# **Proyecto UAO - Piloto Deserción**
## Gerente UAO:  Erik Orozco
## Consultor Innovadata.ai:  Jaime Reinoso

Actores: 
EO. Erik Orozco
JR.  Jaime Reinoso

Formato fechas: YYYY.MM.DD

Hitos Proyecto:
2023.08.30 - Reunion de kickoff:  se explica alcance, proceso a seguir y data posible a capturar.  Powerpoint de soporte.
2023.09.08 - Se recibe email de ERIK con data
2023.09.11 - Jaime analiza data

**Importaciones Globales**


In [1]:
from pandas_profiling import ProfileReport
import pandas as pd

/Users/jaimereinoso/DESARROLLO/UAOPiloto/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/tx/1tx9w9fj6v78n5f12zkwg_vc0000gn/T/ipykernel_99273/3708703399.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


## 1.  Recolección de Datos
Como primera medida se leen los archivos entregados con el fin de entender la data, detectar inconsistencias y validar cantidad de filas disponibles.


### **1.1 Estudiantes**

In [6]:
from pandas_profiling import ProfileReport
import pandas as pd

estudiantes = pd.read_excel('data_original/estudiante.xlsx')
prof = ProfileReport(estudiantes)
prof.to_file(output_file='estudiantest.html')
estudiantes.columns

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 290.54it/s]


Index(['PERIODO', 'ID_ESTUDIANTE', 'IDENTIFICACION', 'PROG', 'FECHA_ADMITIDO',
       'FECHA_DESERTO', 'ESTADO', 'FECHA_GRADO', 'FECHA_NACIMIENTO', 'GENERO',
       'COLEGIO', 'FECHA_GRADO_COLEGIO', 'FECHA_ICFES', 'CIUDAD_COLEGIO'],
      dtype='object')

* (OJO) Se detectan sólo 163 filas.  Posiblemente se requieran más datos.  sin embargo, si el modelo base de MATRICULA y no ESTUDIANTES, se generarían cerca de 9 veces más datos que podrían servir para el piloto.    El modelo por tanto tomaría info de un estudiante que acaba de matricularse durante un nuevo semestre.

* Se detectan 8 PERIODOS desde el 201802 hasta el 202204, lo cual son 4 años.
* Cada estudiante tiene su ID (ok).  
* (OJO) Existe una IDENTIFICACION de cada uno, lo cual no es necesario y es mejor evitar por temas de habeas data.  Será eliminada tanto del excel como del archivo posterior confirmación con ERIK.
* Todos vienen del mismo programa (OK)
* (OJO) hay 85 filas que presentan datos en FECHA_DESERTO.  Frente al total de filas es interesante pues pareciera estar "balanceado", lo que significa que habría cerca de un 50% de deserción!!
* (OJO) frente al estado actual, hay 80 filas activas, 78 inactivas (no cuadra con desercion), y 5 graduados únicamente.
* El género viene balanceado (OK)
* muy pocas instancias de colegios distintos.  Estudiar si hay formas de agruparlos por demografía, geográfia, etc.
* (OJO) Dos campos no traen datos FECHA_GRADO_COLEGIO y FECHA_ICFES
* viene la ciudad del colegio, aunque 117 son de cali y por tanto no hay tanta variabilidad.  Agrupar posiblemente (cali, no-cali).
* NULOS:  de los 163 filas, en FECHA_DESERTO hay datos en 78.  FECHA_GRADO solo en 5.





### **1.2 Admisiones**

In [7]:
admisiones = pd.read_excel('data_original/admisiones.xlsx')
prof = ProfileReport(admisiones)
prof.to_file(output_file='admisiones.html')
admisiones.columns

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 92.63it/s]


Index(['PROGRAMA_CODIGO', 'SEDE_NOMBRE', 'PERIODO_INSCRIPCION',
       'FECHA_INSCRIPCION', 'PROGRAMA_OPCION1_CODIGO', 'NIVELATORIO_INGLES',
       'FECHA_NIVELATORIO_INGLES', 'PRUEBA_DIAGNOSTICA', 'ID_ESTUDIANTE',
       'PERIODO_EXPEDIENTE', 'ACCESO_NOMBRE', 'ESTADO_ACADEMICO',
       'SFDC_ESTADO_ACA', 'CREDITOS_CURSADOS', 'CREDITOS_APROBADOS',
       'FECHA_GRADO', 'RANGO_CREDITOS', 'MEDIO_ENTERO_PROGRAMA',
       'TIPO_SOLICITUD', 'ESTUDIANTE_ARTICULACION', 'PREGUNTA1', 'RESPUESTA1',
       'PREGUNTA2', 'RESPUESTA2', 'PREGUNTA3', 'RESPUESTA3', 'PREGUNTA4',
       'RESPUESTA4', 'PREGUNTA5', 'RESPUESTA5', 'PREGUNTA6', 'RESPUESTA6',
       'EXPEDIENTE_BLOQ_FECHA', 'PROMEDIO_POND_SEM', 'PROMEDIO_POND_ACUM',
       'ULTIMO_PERIODO_MAT', 'CRED_MATRICULADOS', 'ULTIMO_PROCESO_MAT',
       'PILOS_UAO', 'SOLICITUD_PYS_GRADO'],
      dtype='object')

* (OJO) 150 observaciones.  Estudiantes tenía 163.  No todo estudiante tiene info de admisiones.
* todos del mismo programa y misma sede
* NIVELATORIO_INGLES bastante balanceado, con solo 15 missings.
* PRUEBA_DIAGNOSTICA S:89, N:46.  15 missings.   
* ID_ESTUDIANTE:   relación 1:1 entre ADMISIONES y ESTUDIANTES (???)
* ACCESO_NOMBRE:  el tipo de admision (Admision 1er sem, Transferencia interna, Transferencia externa)
* (OJO) ESTADO:  89 activo, 57 inactivo, 6 graduados, 1 Mat (matriculado?).   no cuadra con los datos de estudiante
* (OJO) SFDC_ESTADO_ACA:  no cuadra con ESTADO_ACADEMICO de esta tabla ni con información en ESTUDIANTES.
* (OJO) CREDITOS_CURSADOS:  la cantidad de créditos que ha cursado el estudiante (confirmar). desde 5 hasta 177.    Averiguar datos por semestre (posiblemente en otras tablas)
* (OJO) CREDITOS_APROBADOS:  de 0 a 175.  Interesante.  Ojala conseguir esto por semestre.
* (OJO) FECHA_GRADO.  aparecen 3 instancias.  No cuadra con la info en ESTUDIANTES.
* (OJO) RANGO_CREDITOS.  hay de 10 a 18 y de 16 a 18.  Se sobreponen???
* MEDIO_ENTERO_PROGRAMA: cómo se enteró del programa.  Interesante!!
* (OJO) TIPO_SOLICITUD:  ingreso/transferencia.  Ya cubierta en otra columna.
* (OJO) ESTUDIANTE_ARTICULACIÓN:  NO:140, S:1, NULL: 9  (qué es????   MUY desbalanceado)
* 6 PREGUNTAX / RESPUESTAX.  interesante.
* (OJO) PROMEDIO_POND_SEM:  está en ADMISION? no debería estar en una tabla por semestre?
* (OJO) PROMEDIO_POND_ACUM:  idem al anterior.
* (OJO) CRED_MATRICULADOS: 4 distintos valores.   debería ser por semestre.
* (OJO) ULTIMO_PROCESO_MAT:  ultimo proceso matriculado..  interesante, pero deberia ser mejor por semestre.
* (OJO) PILOS_UAO:  muy desbalanceado.  N: 86, S:1, NULL: 63
* SOLICITUD_PYS_GRADO: interesante.  pero pocas filas lastimósamente. 

IDEA:  centralizar data alrededor del SEMESTRE y no del ESTUDIANTE.  Esto nos aumenta cantidad de filas.   


### **1.3 ESTUDIANTEASIGNATURA**

In [8]:
estudianteasignatura = pd.read_excel('data_original/EstudianteAsignatura.xlsx')
prof = ProfileReport(estudianteasignatura)
prof.to_file(output_file='EstudianteAsignatura.html')
estudianteasignatura.columns

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 486.63it/s]


Index(['PERIODO', 'PERIODO_VERANO', 'ID_ESTUDIANTE', 'COD_DEP', 'COD_ASIG',
       'GRUPO', 'ASIGNATURA-GRUPO', 'CREDITOS', 'CEDULA', 'DEFINITIVA',
       'DEFINITIVA_ALF', 'CANCELADA_VOL', 'CANCELADA_UAO',
       'PERDIDA_INASISTENCIA', 'PERDIDA', 'GANADA'],
      dtype='object')